In [1]:
# Add the libraies path to python
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..\\python\\database'))
sys.path.append(os.path.join(os.getcwd(), '..\\python\\plots'))

# Library imports
import database
import performance_plots

# Constants to use in this notebook
season = "2024"

# Build the database for the season
db = database.database(season)

In [2]:
min_winning_scores = list()

for gw in range(db.get_number_of_gws()):
    gw_scores = db.get_gw_scores(gw)
    gw_scores.sort(reverse=True)
    min_winner = int(len(db.get_managers()) / 2)
    min_winning_scores.append(gw_scores[min_winner-1])

avg_winning_score = sum(min_winning_scores) / len(min_winning_scores)
print("Average minimum winning score: ", avg_winning_score)
print("Average minimum winning player score: ", avg_winning_score / 11)

Average minimum winning score:  147.70833333333334
Average minimum winning player score:  13.428030303030305


In [3]:
manager_best_performing_loss_gw = db.get_manager_best_performing_loss_gws()
print("Best-performing Losers: ")
for manager in manager_best_performing_loss_gw.keys():
    gw = manager_best_performing_loss_gw[manager]
    print(manager, db.get_z_score(manager, gw), db.get_gw_score(manager, gw))

print()    

manager_worst_performing_win_gws = db.get_manager_worst_performing_win_gws()
print("Worst-performing Winners: ")
for manager in manager_worst_performing_win_gws.keys():
    gw = manager_worst_performing_win_gws[manager]
    print(manager, db.get_z_score(manager, gw), db.get_gw_score(manager, gw))

Best-performing Losers: 
zac 0.16924423694193202 153.0
morfin 0.7704304429195254 166.5
sean 1.2180930930930927 168.5
aj 0.06285102968708195 153.5
katabe 0.2500612895317477 166.0
ben -0.06564551422319474 156.5
matt 0.6774193548387093 180.0
tj 0.41053757689221704 166.5
rafa 0.2501583280557308 160.0
vu 0.3937626205968143 187.0

Worst-performing Winners: 
zac -0.5780182232346245 113.0
morfin -0.7373913043478257 107.0
sean -0.7214228617106314 102.5
aj -1.033269353806782 104.0
katabe -0.08106543138390306 121.0
ben -0.47723019338739897 126.5
matt -0.3639846743295019 122.5
tj 0.1623006833712981 126.0
rafa -0.7203690560681338 123.0
vu 0.10535307517084251 125.0


In [4]:
def get_manager_performance_for_gw(database, manager, best_performance, won_gw):
    gw_to_return = -1
    manager_z_score = -100 if best_performance else 100
    for gw in range(database.get_number_of_gws()):
        opponent = database.get_opponent(manager, gw)
        manager_score = database.get_gw_score(manager, gw)
        opponent_score = database.get_gw_score(opponent, gw)
        # Check if the result matches the parameter
        if((manager_score > opponent_score) == won_gw):
            manager_gw_z_score = database.get_z_score(manager, gw)
            if((best_performance and manager_gw_z_score > manager_z_score) or
               (not best_performance and manager_gw_z_score < manager_z_score)):
                manager_z_score = manager_gw_z_score
                gw_to_return = gw
    return gw_to_return
    
def get_manager_performance_gws(database, best_performance, won_gw):
    manager_results = dict()
    for manager in database.get_managers():
        manager_results[manager] = get_manager_performance_for_gw(database, manager, best_performance, won_gw)
    return manager_results

def get_best_performing_loss_gws(database):
    return get_manager_performance_gws(database, True, False)

def get_worst_performing_win_gws(database):
    return get_manager_performance_gws(database, False, True)

def get_best_performing_win_gws(database):
    return get_manager_performance_gws(database, True, True)

def get_worst_performing_loss_gws(database):
    return get_manager_performance_gws(database, False, False)

print(get_best_performing_loss_gws(db))
print(get_worst_performing_win_gws(db))
print(get_best_performing_win_gws(db))
print(get_worst_performing_loss_gws(db))

{'zac': 29, 'morfin': 35, 'sean': 0, 'aj': 1, 'katabe': 3, 'ben': 34, 'matt': 8, 'tj': 3, 'rafa': 34, 'vu': 23}
{'zac': 34, 'morfin': 1, 'sean': 8, 'aj': 12, 'katabe': 19, 'ben': 27, 'matt': 29, 'tj': 4, 'rafa': 26, 'vu': 11}
{'zac': 4, 'morfin': 0, 'sean': 27, 'aj': 5, 'katabe': 26, 'ben': 10, 'matt': 7, 'tj': 17, 'rafa': 4, 'vu': 32}
{'zac': 0, 'morfin': 28, 'sean': 15, 'aj': 0, 'katabe': 34, 'ben': 11, 'matt': 27, 'tj': 13, 'rafa': 29, 'vu': 24}


In [22]:
player_id_data_map = db.get_player_id_map()
manager_count_player_ids = dict()

max_manager_count = 0

for player_id in player_id_data_map.keys():
    manager_count = len(player_id_data_map[player_id])
    if(max_manager_count < manager_count):
        max_manager_count = manager_count
    if(manager_count not in manager_count_player_ids.keys()):
        manager_count_player_ids[manager_count] = list()
    manager_count_player_ids[manager_count].append(player_id)

league_bikes = list()
for player_id in manager_count_player_ids[max_manager_count]:
    league_bikes.append(player_id_data_map[player_id][0].get_name())
print(len(player_id_data_map))
print(league_bikes)

341
['Sepp van den Berg', 'Conor Bradley', 'Cody Gakpo', 'Illia Zabarnyi', 'Alejandro Garnacho', 'John McGinn', 'Kevin Schade', 'Darwin Nunez']


In [28]:
for manager in db.get_managers():
    player_data = db.get_manager_player_database(manager).get_all_player_data()
    total_score = 0
    for player_datum in player_data:
        total_score += player_datum.get_points()
    print(manager +"'s total score: " + str(total_score))
    most_contributive_player
    for player_datum in player_data:
        total_score += player_datum.get_points()

zac's total score: 5303.5
morfin's total score: 4618.5
sean's total score: 5052.0
aj's total score: 4652.5
katabe's total score: 5045.0
ben's total score: 5023.0
matt's total score: 5271.0
tj's total score: 5728.5
rafa's total score: 5038.0
vu's total score: 5847.0


In [1]:
from fantraxapi import FantraxAPI

In [3]:

league_id = "96igs4677sgjk7ol"

api = FantraxAPI(league_id)

In [4]:
for _, scoring_period in api.scoring_periods().items():
    print("")
    print(scoring_period)

Unauthorized: Unauthorized: Not Logged in

In [5]:
fpl_23 = "kfszz7vdmdl0krou"
fpl_24 = "962c1utjlzbm8vb8"
fpl_25 = "ablch5snljylssmy"

AttributeError: 'ScoringPeriod' object has no attribute 'week'

In [ ]:
import pickle
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920,1600")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36")

with webdriver.Chrome(service=service, options=options) as driver:
    driver.get("https://www.fantrax.com/login")
    time.sleep(30)
    pickle.dump(driver.get_cookies(), open("fantraxloggedin.cookie", "wb"))

In [ ]:
import pickle
from fantraxapi import FantraxAPI
from requests import Session

session = Session()

with open("fantraxloggedin.cookie", "rb") as f:
    for cookie in pickle.load(f):
        session.cookies.set(cookie["name"], cookie["value"])
        
api = FantraxAPI(fpl_24)
print(api.trade_block()) 

AttributeError: 'ScoringPeriod' object has no attribute 'week'